In [ ]:
from typing import Tuple, List

import tensorflow as tf
from tensorflow import keras

import numpy as np

In [ ]:
import sys

LIB_PATH = '/content/drive/MyDrive/GSC/GSC_helper'
sys.path.append(LIB_PATH)

from BCResNet_tf import BCResNet
from utils import unzipzip, zipzip
from GSC import download_GSC
from GSC12 import SpeechCommands12

In [ ]:
#input = np.random.rand(128, 40, 101, 1)
model = BCResNet(1,
                   12,
                   False,
                    3)
#model(input).shape

In [ ]:
model.build(input_shape = (1, 40, 101, 1))
model.summary()

Model: "bc_res_net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_28 (Sequential)  (1, 12)                   55224     
                                                                 
Total params: 55224 (215.72 KB)
Trainable params: 51072 (199.50 KB)
Non-trainable params: 4152 (16.22 KB)
_________________________________________________________________


## Try to convert to TFLite

In [ ]:
ZIP_MAP = download_GSC('https://drive.google.com/file/d/1-8quY_z264H0kgWqbrQPWQIvuAfioBLh/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-B1vD4fz2kZR9It2xBmq1PJ0afHnaJVg/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-CFA8mlCdT4lgnIuX4-S_Vs0ATtRBbSo/view?usp=drive_link',
                       '/content/GSC_12',
                       end = '.zip')
CSV_MAP = download_GSC('https://drive.google.com/file/d/1-DYeiWPis6npYe8Z22Sa38nB2RdokMWH/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-FB8YPbdvL2Vrhur94nWOGHL2M7RwzW0/view?usp=drive_link',
                       'https://drive.google.com/file/d/1-H7ZkCrzEl9VxfdOuH5YmCNSC6YP8CFw/view?usp=drive_link',
                       '/content/GSC_12',
                       end = '.csv')

Downloading...
From (original): https://drive.google.com/uc?id=1-8quY_z264H0kgWqbrQPWQIvuAfioBLh
From (redirected): https://drive.google.com/uc?id=1-8quY_z264H0kgWqbrQPWQIvuAfioBLh&confirm=t&uuid=1d54cac8-0f30-4cdf-8c34-b3bbbf5e9a7c
To: /content/GSC_12/train.zip
100%|██████████| 2.18G/2.18G [00:19<00:00, 113MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-B1vD4fz2kZR9It2xBmq1PJ0afHnaJVg
From (redirected): https://drive.google.com/uc?id=1-B1vD4fz2kZR9It2xBmq1PJ0afHnaJVg&confirm=t&uuid=3e42ad95-a1b8-42f9-9cc9-612a3ae9afec
To: /content/GSC_12/val.zip
100%|██████████| 64.6M/64.6M [00:00<00:00, 141MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1-CFA8mlCdT4lgnIuX4-S_Vs0ATtRBbSo
From (redirected): https://drive.google.com/uc?id=1-CFA8mlCdT4lgnIuX4-S_Vs0ATtRBbSo&confirm=t&uuid=ae682264-0ef7-4f82-a19d-4efb3dcef65b
To: /content/GSC_12/test.zip
100%|██████████| 65.7M/65.7M [00:00<00:00, 132MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-DYei

In [ ]:
import pandas as pd
import os

class SC_12(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self,
                 root: str,
                 zip_map: dict,
                 csv_map: dict,
                 unzip: bool = True,
                 subset: str = 'train',
                 batch_size: int = 32,
                 shuffle: bool = True):
        'Initialization'
        super().__init__()
        local_path = os.path.join(root, subset)
        self.root = root
        if not os.path.exists(local_path):
            os.mkdir(local_path)
            unzipzip(zip_map[subset], local_path)
        if unzip:
            unzipzip(zip_map[subset], local_path)
        self.csv = pd.read_csv(csv_map[subset])
        self.subset = subset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the numbber of batches per epoch'
        return int(np.floor(len(self.csv)/self.batch_size))

    def __getitem__(self, index):
        'Generate on batch of data'
        # Generate indexes of the batcch
        indexes = self.indexes[index*self.batch_size: (index+1)*self.batch_size]

        # Generate data
        X, y = self.__data_generation(indexes)

        return X, y

    def on_epoch_end(self):
        'Undates indexes after each epoch'
        self.indexes = np.arange(len(self.csv))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X: (n_samples, *dim, n_channels)
        # Initialization
        X = []
        y = []

        # Generate data
        for i in indexes:
            # Store sample
            row = self.csv.iloc[i]
            X.append(np.load(os.path.join(self.root, row['link']))['arr_0'])

            # Store class
            y.append(row['label'])
        X = np.stack(X, axis = 0)
        y = np.stack(y, axis = 0)
        return tf.convert_to_tensor(X)[..., tf.newaxis], tf.convert_to_tensor(y)

In [ ]:
train_dataloader = SC_12('/content/GSC_12', ZIP_MAP, CSV_MAP, unzip = False, subset = 'train', batch_size = 128, shuffle = True)
val_dataloader = SC_12('/content/GSC_12', ZIP_MAP, CSV_MAP, unzip = False, subset = 'val', batch_size = 128, shuffle = False)
test_dataloader = SC_12('/content/GSC_12', ZIP_MAP, CSV_MAP, unzip = False, subset = 'test', batch_size = 10, shuffle = False)

Extracted /content/GSC_12/train.zip
Extracted /content/GSC_12/val.zip
Extracted /content/GSC_12/test.zip


In [ ]:
x, y = next(iter(val_dataloader))
x.shape

TensorShape([128, 40, 101, 1])

In [ ]:
model(x).shape

TensorShape([128, 12])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

model.fit(train_dataloader, validation_data = test_dataloader, epochs = 1)

Epoch 1/3
1153/1153 [==============================] - 219s 176ms/step - loss: 0.3754 - accuracy: 0.8745 - val_loss: 0.3338 - val_accuracy: 0.9172
Epoch 2/3
1153/1153 [==============================] - 198s 172ms/step - loss: 0.3324 - accuracy: 0.8879 - val_loss: 0.2129 - val_accuracy: 0.9329
Epoch 3/3
1153/1153 [==============================] - 198s 172ms/step - loss: 0.3048 - accuracy: 0.8967 - val_loss: 0.6599 - val_accuracy: 0.8601


In [ ]:
from tqdm import tqdm

test_specs = []
test_labels = []

for i in tqdm(range(len(test_dataloader.csv))):
    row = test_dataloader.csv.iloc[i]
    spec = np.load(os.path.join('/content/GSC_12', row['link']))['arr_0']
    test_specs.append(spec)
    test_labels.append(row['label'])

test_specs = tf.convert_to_tensor(np.stack(test_specs, axis = 0))[..., tf.newaxis]
test_labels = tf.convert_to_tensor(np.stack(test_labels))

100%|██████████| 4890/4890 [00:02<00:00, 1632.01it/s]


In [ ]:
model.save('bcresnet3_tf.keras')

In [ ]:
def representative_data_gen():
    for input_value in tf.data.Dataset.from_tensor_slices(test_specs).batch(1).take(100):
        yield [input_value]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint 8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


In [ ]:
net = keras.models.load_model('/content/bcresnet3_tf.keras')
net.summary()

Model: "bc_res_net_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_57 (Sequential)  (1, 12)                   55224     
                                                                 
Total params: 55224 (215.72 KB)
Trainable params: 51072 (199.50 KB)
Non-trainable params: 4152 (16.22 KB)
_________________________________________________________________


In [ ]:
import pathlib

tflite_models_dir = pathlib.Path("/tmp/gsc_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the dynamic range quantized model:
tflite_model_quant_file = tflite_models_dir/"gsc_tflite_model_quant2.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

214504

In [ ]:
import shutil
shutil.copy2(str(tflite_model_quant_file), '/content')

'/content/mnist_model_quant3.tflite'

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [ ]:
tflite_model_file = tflite_models_dir/"gsc_model.tflite"
tflite_model_file.write_bytes(tflite_model)

300552

In [ ]:
# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
    global test_specs

    # Initialize the interpreter
    interpreter = tf.lite.Interpreter(model_path = str(tflite_file))
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    predictions = np.zeros((len(test_image_indices), ), dtype = int)
    for i, test_image_index in enumerate(test_image_indices):
        test_image = test_specs[test_image_index]

        # Check if the input type is quantized, the rescale input data to to uint8
        if input_details['dtype'] == np.uint8:
            input_scale, input_zero_point = input_details['quantization']
            test_image = test_image/input_scale + input_zero_point

        test_image = np.expand_dims(test_image, axis = 0).astype(input_details['dtype'])
        interpreter.set_tensor(input_details['index'], test_image)
        interpreter.invoke()
        output = interpreter.get_tensor(output_details['index'])[0]

        predictions[i] = output.argmax()

    return predictions

# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global test_specs
  global test_labels

  test_image_indices = range(test_specs.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(test_labels== predictions) * 100) / len(test_specs)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_specs)))

In [ ]:
evaluate_model(tflite_model_quant_file, model_type="Quantized")

Quantized model accuracy is 79.4683% (Number of test samples=4890)


In [ ]:
interpreter = tf.lite.Interpreter(model_path = str(tflite_model_quant_file))
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
print(input_details)
print(output_details)

{'name': 'serving_default_input_1:0', 'index': 0, 'shape': array([  1,  40, 101,   1], dtype=int32), 'shape_signature': array([ -1,  40, 101,   1], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.5479903817176819, 182), 'quantization_parameters': {'scales': array([0.5479904], dtype=float32), 'zero_points': array([182], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
{'name': 'StatefulPartitionedCall:0', 'index': 293, 'shape': array([12], dtype=int32), 'shape_signature': array([12], dtype=int32), 'dtype': <class 'numpy.uint8'>, 'quantization': (0.1329231858253479, 128), 'quantization_parameters': {'scales': array([0.13292319], dtype=float32), 'zero_points': array([128], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}


In [ ]:
evaluate_model(tflite_model_file, model_type="Float32")

Float32 model accuracy is 91.0020% (Number of test samples=4890)


In [ ]:
import shutil
shutil.copy2(str(tflite_model_quant_file), '/content')

'/content/gsc_tflite_model_quant2.tflite'